<a href="https://colab.research.google.com/github/GeoffreyKimani/InvetorySystem-VueJS-/blob/master/house_expenditure_anaylsis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install PyDrive

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
# https://docs.google.com/spreadsheets/d/1EmoHMuC15JxaXORjtYBFe0ybSEIAbrlbmzFVha8_fMs/edit?usp=sharing
downloaded = drive.CreateFile({'id':"1EmoHMuC15JxaXORjtYBFe0ybSEIAbrlbmzFVha8_fMs"})
downloaded.GetContentFile('household_expenditure', mimetype='application/vnd.openxmlformats-officedocument.spreadsheetml.sheet')

In [ ]:
data = pd.read_excel('household_expenditure')

# select only columns of interest
data = data.loc[:, 'Purchaser':'Misc Cost']

# remove rows with no entries for all fields
data.dropna(axis=0, how='all', inplace=True)

data

,Purchaser,Date,Cost,Category,Item,Month,Food Cost,Misc Cost
0,GK,2021-08-30 00:00:00,1600.0,Food,Sardines,Aug 2021,1600.0,0.0
1,NK,2021-09-01 00:00:00,4300.0,Food,Sardines 4 * 700 Milk 3 * 500,Sep 2021,4300.0,0.0
2,GK,2021-09-02 00:00:00,11400.0,House Items,2 Fantas + Nescafe coffee + 2 Instant porriage,Sep 2021,0.0,11400.0
3,GK,2021-09-02 00:00:00,1400.0,Food,Bread,Sep 2021,1400.0,0.0
4,NK,2021-09-02 00:00:00,3000.0,Food,1 KG Beef,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
74,MX,2021-11-18 00:00:00,6400.0,Food,"Kaunga 5kg-2600, Sardines 3-2400, Ubugari 1kg-...",Nov 2021,6400.0,0.0
75,MX,2021-11-19 00:00:00,10600.0,Food,"Cornflakes -8500, Indomie -2100",Nov 2021,10600.0,0.0
76,GK,19/11/2021,2700.0,Food,milk -2 pacs @600 each soda @1500,Nov 2021,2700.0,0.0
77,MX,2021-11-22 00:00:00,12400.0,Food,"Milk-6000, Groceries-66",Nov 2021,12400.0,0.0


In [ ]:
# select all entries where names are unique and form their own dataframes
name_list = ['NK', 'MX', 'JO', 'GK']
purchaser_list = {}

for name in name_list:
    new_df = name + '_df'
    new_df = data[data['Purchaser'] == name]
    purchaser_list.update({name: new_df})

purchaser_list

{'GK':    Purchaser                 Date     Cost  ...     Month Food Cost Misc Cost
 0         GK  2021-08-30 00:00:00   1600.0  ...  Aug 2021    1600.0       0.0
 2         GK  2021-09-02 00:00:00  11400.0  ...  Sep 2021       0.0   11400.0
 3         GK  2021-09-02 00:00:00   1400.0  ...  Sep 2021    1400.0       0.0
 5         GK  2021-09-04 00:00:00  12400.0  ...  Sep 2021       0.0   12400.0
 6         GK  2021-09-04 00:00:00   3200.0  ...  Sep 2021    3200.0       0.0
 11        GK  2021-09-12 00:00:00  14000.0  ...  Sep 2021   14000.0       0.0
 13        GK           13/09/2021   3200.0  ...  Sep 2021    3200.0       0.0
 14        GK           15/09/2021  11600.0  ...  Sep 2021   11600.0       0.0
 15        GK           17/09/2021   5300.0  ...  Sep 2021    5300.0       0.0
 19        GK           26/09/2021   9500.0  ...  Sep 2021    9500.0       0.0
 21        GK           30/09/2021   3200.0  ...  Sep 2021    3200.0       0.0
 23        GK           03/10/2021   2000.0  .

In [ ]:
# total expenses
purchaser_expense = {}
for purchaser, df in purchaser_list.items():
    purchaser_expense.update({purchaser: df['Cost'].sum()})

purchaser_expense

{'GK': 189800.0, 'JO': 90930.0, 'MX': 267250.0, 'NK': 7300.0}

#Share Contribution and Deficit

In [ ]:
total_expense = sum(purchaser_expense.values())
total_expense

555280.0

In [ ]:
# personal share
personal_share = total_expense/len(purchaser_expense)
personal_share

138820.0

In [ ]:
# deficit
deficits = {person: contribution - personal_share for person, contribution in purchaser_expense.items()}
deficits

{'GK': 50980.0, 'JO': -47890.0, 'MX': 128430.0, 'NK': -131520.0}

# Monthly Analysis 💰

In [ ]:
# plt.bar(total_expense.keys(), total_expense.values())